In [11]:
import numpy as np
import os
import torch
from misc.example_helper import *
import importlib
import processing
import models
%load_ext autoreload
%autoreload 2
importlib.reload(processing)
importlib.reload(models)
from processing.parametric_data_manager import ParametricSHREDDataManager
from models.shred_models import SHRED

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os

# Initialize ParametricSHREDDataManager
# manager = ParametricSHREDDataManager(
#     lags = 20,
#     train_size = 0.8,
#     val_size = 0.1,
#     test_size = 0.1,
#     compression = 50,
#     )

# Add data to manager (with sensors)
input_folder = "kuramoto_sivashinsky"
filename = 'KuramotoSivashinsky_data.npz'
file_path = os.path.join(input_folder, filename)
dataset = np.load(file_path)

# dataset = np.load('kuramoto_sivashinsky\KuramotoSivashinsky_data.npz')
data = dataset['u'] # shape (500, 201, 100)
mu = dataset['mu'] # shape (500, 201, 2)
mobile_sensors = [
    forward_backward_walk(start=0, end = data.shape[2], timesteps=data.shape[1], forward_first=True),
    forward_backward_walk(start=0, end = data.shape[2], timesteps=data.shape[1], forward_first=False),
]

manager.add(
    data=data,
    random_sensors=2,
    stationary_sensors=[(0,), (1,)],
    # mobile_sensors=mobile_sensors,
    params=mu,
    id = 'DS'
)


hello


In [14]:
#seperate sensors from params, so when user enters new data, it is clear what the sensor order is and what the param order is when there are sensor data and param data from multiple fields
# do multiple fields make sense in parametric shred? 
# can different fields have different params? the result of 'mixing up' params of one field and keeping another field params constant to see effect??
manager.sensor_measurements

,trajectory,KS-0,KS-1,KS-2,KS-3,DS-0,DS-1,DS-2,DS-3
0,0,1.000305,-1.053229,1.000000,1.093962,-0.001146,0.864778,1.000000,1.093962
1,0,1.272962,-0.980285,0.376289,0.546796,0.000188,1.152519,0.376289,0.546796
2,0,1.335450,-0.962610,0.450057,0.619256,0.015522,1.211298,0.450057,0.619256
3,0,1.411313,-0.934613,0.511200,0.682191,0.018314,1.284031,0.511200,0.682191
4,0,1.495320,-0.912607,0.556852,0.728624,0.014118,1.371218,0.556852,0.728624
...,...,...,...,...,...,...,...,...,...
100495,499,1.155829,-1.632350,-0.658349,-0.446285,-0.098147,1.109019,-0.658349,-0.446285
100496,499,0.806459,-1.131872,-0.602322,-0.400786,0.034844,0.633907,-0.602322,-0.400786
100497,499,-0.303753,-0.694939,-0.181651,-0.137417,0.154053,-0.541562,-0.181651,-0.137417
100498,499,-1.805045,-0.363572,0.561369,0.286482,0.263992,-1.939818,0.561369,0.286482


In [15]:
manager.sensor_summary

,field id,sensor id,sensor type,location/trajectory
0,KS,KS-0,stationary (randomly selected),"(10,)"
1,KS,KS-1,stationary (randomly selected),"(31,)"
2,KS,KS-2,stationary (user selected),"(0,)"
3,KS,KS-3,stationary (user selected),"(1,)"
4,DS,DS-0,stationary (randomly selected),"(46,)"
5,DS,DS-1,stationary (randomly selected),"(11,)"
6,DS,DS-2,stationary (user selected),"(0,)"
7,DS,DS-3,stationary (user selected),"(1,)"


In [16]:
manager.params

,trajectory,KS param 0,KS param 1,DS param 0,DS param 1
0,0,1.423285,1.568899,1.423285,1.568899
1,0,1.423285,1.568899,1.423285,1.568899
2,0,1.423285,1.568899,1.423285,1.568899
3,0,1.423285,1.568899,1.423285,1.568899
4,0,1.423285,1.568899,1.423285,1.568899
...,...,...,...,...,...
100495,499,1.160304,1.782302,1.160304,1.782302
100496,499,1.160304,1.782302,1.160304,1.782302
100497,499,1.160304,1.782302,1.160304,1.782302
100498,499,1.160304,1.782302,1.160304,1.782302


In [17]:
# Get train/val/test datasets
train_set, val_set, test_set = manager.preprocess()

# # Print dataset shapes
print('Data Shapes:')
print ('Reconstructor Data')
print('train X:', train_set.reconstructor_dataset.X.shape)
print('train Y:', train_set.reconstructor_dataset.Y.shape)
print('val X:', val_set.reconstructor_dataset.X.shape)
print('val Y:', val_set.reconstructor_dataset.Y.shape)
print('test X:', test_set.reconstructor_dataset.X.shape)
print('test Y:', test_set.reconstructor_dataset.Y.shape)

Data Shapes:
Reconstructor Data
train X: torch.Size([80400, 21, 12])
train Y: torch.Size([80400, 100])
val X: torch.Size([10050, 21, 12])
val Y: torch.Size([10050, 100])
test X: torch.Size([10050, 21, 12])
test Y: torch.Size([10050, 100])


In [18]:
result = manager.postprocess(train_set.reconstructor_dataset.Y)
print(result['KS'].shape)

(400, 201, 100)


In [26]:
# initialize SHRED
shred = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
val_errors = shred.fit(train_set, val_set, num_epochs=100, patience=100, verbose=True, batch_size = 1024)


Fitting Reconstructor...


Epoch 1/100:   0%|          | 0/79 [00:00<?, ?batch/s]

Epoch 2/100:   0%|          | 0/79 [00:00<?, ?batch/s]

Epoch 3/100:   0%|          | 0/79 [00:00<?, ?batch/s]

Epoch 4/100:   0%|          | 0/79 [00:00<?, ?batch/s]

...

Epoch 96/100:   0%|          | 0/79 [00:00<?, ?batch/s]

Epoch 97/100:   0%|          | 0/79 [00:00<?, ?batch/s]

Epoch 98/100:   0%|          | 0/79 [00:00<?, ?batch/s]

Epoch 99/100:   0%|          | 0/79 [00:00<?, ?batch/s]

Epoch 100/100:   0%|          | 0/79 [00:00<?, ?batch/s]

In [29]:
reconstruction = shred.reconstruct(test_set.reconstructor_dataset.X)
reconstruction.shape

(10050, 100)

In [ ]:
### given sensor measurements of shape (ntraj, ntimes, nsensors) -> (ntraj*ntimes, lags, nsensors) and params of (ntraj, ntimes, nparams) -> (ntaj*ntimes, lags, nsensors)


In [ ]:
# development testing
train_set.reconstructor.X[201,:,0]

In [ ]:
# NEW DATA
# n_traj, n_times, nsensors
# n_traj, n_times, nparams


